In [10]:
import cv2
import numpy as np

In [11]:
# 读取图片
image1 = cv2.imread(r"C:\Users\28106\Desktop\1.jpg")
image2 = cv2.imread(r"C:\Users\28106\Desktop\2.jpg")

In [12]:
# 转换为灰度图像
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

In [13]:
# 使用SIFT检测关键点和描述符
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

In [14]:
# 使用FLANN匹配器进行特征匹配
flann_index_kdtree = 1
index_params = dict(algorithm=flann_index_kdtree, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

In [15]:
# 保留良好的匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)


In [16]:
# 获取匹配点对的坐标
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

In [17]:
# 计算单应性矩阵
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# 应用单应性变换
result = cv2.warpPerspective(image1, H, (image2.shape[1], image2.shape[0]))
print(H)

[[ 2.80035023e-01 -1.13114300e+00  2.79842478e+03]
 [ 4.27558210e-01  7.11543334e-01 -3.96005237e+02]
 [-1.41467355e-04  1.25412726e-04  1.00000000e+00]]


In [18]:
# 显示结果
cv2.imwrite(r"C:\Users\28106\Desktop\result.jpg", result)
cv2.waitKey(0)
cv2.destroyAllWindows()